In [1]:
# Access the autogen libraries.
import sys
sys.path.append('..')

# Trial wave functions and QMC job management.
This notebook walks through the process of building a QWalk job. 
A few notes:
* A QWalk writer, reader, and runner will work exactly the same as corresponding PySCF objects.
* A QWalk manager will handle waiting for other runs to finish, and converting the files into a trial wave function.

### Setting up the trial wave function calculation.
For the hydrogen run, let's optimize a Slater-Jastrow wave function with QWalk variance optimization.
First step is setting up the mean field calculation.

In [2]:
from autopyscf import PySCFWriter,PySCFReader
from autorunner import PySCFRunnerLocal
from manager import PySCFManager

h2='\n'.join([
    'H 0.0 0.0 0.0 ',
    'H 0.74 0.0 0.0 '
])
path='05-scratch'

# This is covered in 04-job_management.
pyscf_manager = PySCFManager(
    path=path,
    name='h2_pbe',
    writer=PySCFWriter({
        'xyz':h2,
        'method':'RKS',
        'dft':'pbe,pbe'}),
    runner=PySCFRunnerLocal()
)

The PySCF calculation is defined, but so far no calculations have been started.

QWalk writers can take explicit trial wave function input (as a string), but autogen can also handle it for you.
Make a `TrialFunc` object with the manager you want to generate the trial wave function.

In [3]:
from trialfunc import SlaterJastrow
sj=SlaterJastrow(pyscf_manager)

### Setting up the QWalk run.
Now we can set up the corresponding objects for QWalk.

In [4]:
from variance import VarianceWriter,VarianceReader
from autorunner import RunnerLocal
from manager import QWalkManager
from imp import reload
import time

vopt_writer = VarianceWriter({'iterations':6})

vopt_manager = QWalkManager(
    name='h2_vopt',
    path=path,
    writer=vopt_writer,
    runner=RunnerLocal(np=4),
    reader=VarianceReader(),
    trialfunc=sj # This is new!
)
while not vopt_manager.completed:
    vopt_manager.nextstep()

QWalkManager@05-scratch/h2_vopt : next step.
QWalkManager@05-scratch/h2_vopt : checking trial function.
PySCFManager@05-scratch/h2_pbe : next step.
PySCFManager@05-scratch/h2_pbe : h2_pbe status= not_started
PySCFRunnerLocal : executed python3 h2_pbe.py > h2_pbe.py.o
VarianceWriter : Trial function not ready. Postponing input file generation.
QWalkManager@05-scratch/h2_vopt : h2_vopt status= not_started
QWalkManager@05-scratch/h2_vopt : next step.
QWalkManager@05-scratch/h2_vopt : checking trial function.
PySCFManager@05-scratch/h2_pbe : next step.
PySCFManager@05-scratch/h2_pbe : h2_pbe status= ready_for_analysis
PySCFManager@05-scratch/h2_pbe : h2_pbe status= done, task complete.
PySCFManager@05-scratch/h2_pbe : h2_pbe generating QWalk files.
QWalkManager@05-scratch/h2_vopt : h2_vopt status= not_started
QWalkManager@05-scratch/h2_vopt : h2_vopt status= submitted
RunnerLocal : executed mpirun -n 4 /home/brian/bin/qwalk h2_vopt.variance &> h2_vopt.variance.out
QWalkManager@05-scratch/h

So what just happened here? 
After setting up the calculation, we called `nextstep` repeatedly and the QWalk manager:
* Noticed the PySCF run was not complete, and ran the PySCF run.
* The PySCF run exported the qwalk files needed by QWalk.
* The `QWalkManager` generated its imput files using the files from PySCF.
* `QWalkManager` ran QWalk and collected the QWalk results.

The TrialFunc objects are meant to be modular and flexible. 
You can build TrialFunc objects in a variety of ways, which the next notebook will demonstrate.

# Next: building different trial functions (06-trialfunctions.ipynb).